In [1]:
import pandas as pd 
import json
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import os
import collections
import pickle as pkl
import numpy as np

In [2]:
# with open(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\fast_abs\finished_files\test\2000.json') as f:
#     data = json.load(f)

In [3]:
mode = 'valid'

In [59]:
# df = pd.read_csv(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\data\raw_data\data\{}.csv'.format(mode) ,index_col=0)
df = pd.read_csv(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\data\{}.csv'.format(mode),index_col=0)
df.head()

,summary,fullText
0,Bản_án cho đối_tượng giả_danh công_an để lừa_đảo,"Ngày 25/2 , TAND TP. Đà_Nẵng tuyên_phạt Hồ_Xuâ..."
1,Nam thanh_niên thủ_dâm trên xe_buýt từng bị xử...,Phát_hiện nam thanh_niên đang thủ_dâm trên xe_...
2,"Không được công_nhận , thuốc "" sinh con theo ý...",“ Baby_Support ” và “ Hello baby ” là hai cái ...
3,VEC từ_chối phục_vụ vĩnh_viễn 2 ô_tô gây_rối t...,Liên_quan đến vụ gây_rối tại trạm thu phí trên...
4,VKSND tỉnh Sóc_Trăng xin_lỗi oan sai “ nữ_hoàn...,Bà Huỳnh_Ngọc_Bích được mọi người biết đến là ...


# BERTSUM

In [6]:
def token(text):
    text = re.sub('Th .', 'Th.', text)
    text = re.sub('\.', '', text)
    text = re.sub('  ', ' . ', text)
    para = []
    sent_token = sent_tokenize(text)
    for sent in sent_token:
        word_token = word_tokenize(sent)
        para.append(word_token)
    return para

In [7]:
df = df.dropna()
df.isnull().sum()

summary     0
fullText    0
dtype: int64

In [8]:
df['fullText_token'] = df['fullText'].apply(lambda text: token(text))
df['summary_token'] = df['summary'].apply(lambda text: token(text))

In [9]:
df = df.reset_index()
df.tail()

,index,summary,fullText,fullText_token,summary_token
22637,22637,TP HCM xây cầu 500 tỷ qua đảo Kim_Cương,"Dài gần 300 m , cầu_nối đại_lộ Mai_Chí_Thọ với...","[[Dài, gần, 300, m, ,, cầu_nối, đại_lộ, Mai_Ch...","[[TP, HCM, xây, cầu, 500, tỷ, qua, đảo, Kim_Cư..."
22638,22638,Miễn học_phí cho 16.000 học_sinh vùng_biển Quả...,16.000 học_sinh các cấp ở 4 huyện vùng_biển Qu...,"[[16000, học_sinh, các, cấp, ở, 4, huyện, vùng...","[[Miễn, học_phí, cho, 16000, học_sinh, vùng_bi..."
22639,22639,TP HCM yêu_cầu trường công_lập nhận các trẻ bị...,Giám_đốc Sở Giáo_dục TP HCM yêu_cầu các trường...,"[[Giám_đốc, Sở, Giáo_dục, TP, HCM, yêu_cầu, cá...","[[TP, HCM, yêu_cầu, trường, công_lập, nhận, cá..."
22640,22640,"Rơi thang_máy công_trình cao_tầng , 7 người bị...",Máy vận thăng đang từ tầng 6 một công_trình ở ...,"[[Máy, vận, thăng, đang, từ, tầng, 6, một, côn...","[[Rơi, thang_máy, công_trình, cao_tầng, ,, 7, ..."
22641,22641,Nghị_trường mổ_xẻ 5 dự_án nghìn tỷ ' đắp chiếu ',Nhiều đại_biểu Quốc_hội bức_xúc vì 5 dự_án ngh...,"[[Nhiều, đại_biểu, Quốc_hội, bức_xúc, vì, 5, d...","[[Nghị_trường, mổ_xẻ, 5, dự_án, nghìn, tỷ, ', ..."


In [10]:
path = r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\BertSum\data\{}'.format(mode)
dic = []
i = 0
for ind, row in df.iterrows():
    ind = row['index']
    full = row['fullText_token']
    summary = row['summary_token']
    dic.append({'src': full, 'tgt':summary})
    
    if (ind+1) % 2000 == 0:
        with open(os.path.join(path, 'vnds.{}.{}.json'.format(mode, i)), 'w') as f:
            json.dump(dic, f)
        i+=1
        dic = []
    if (ind+1) == len(df):
        with open(os.path.join(path, 'vnds.{}.{}.json'.format(mode, i)), 'w') as f:
            json.dump(dic, f)

# fastAbs

In [11]:
# def sent_token(text):
#     text = re.sub('Th .', 'Th.', text)
#     text = re.sub('\.', '', text)
#     text = re.sub('  ', ' . ', text)
#     sent_token = sent_tokenize(text)
    
#     return sent_token

In [12]:
# df = df.dropna()

In [13]:
# df.isnull().sum()

In [14]:
# df['fullText_token'] = df['fullText'].apply(lambda x: sent_token(x))
# df['summary_token'] = df['summary'].apply(lambda x: sent_token(x))

In [15]:
# df = df.reset_index()

In [16]:
# if mode == 'train':
#     vocab_counter = collections.Counter()

# path = r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\data\fastAbs_data\{}'.format(mode)
# def write_json(row):
#     ind = row['index']
#     full = row['fullText_token']
#     summary = row['summary_token']
#     dic = {'id':ind, 'article': full, 'abstract':summary}
#     # with open(os.path.join(path, '{}.json'.format(ind)), 'w') as f:
#     #     json.dump(dic, f)

#     if mode == 'train':
#         art_tokens = ' '.join(full).split()
#         abs_tokens = ' '.join(summary).split()
#         tokens = art_tokens + abs_tokens
#         tokens = [t.strip() for t in tokens] # strip
#         tokens = [t for t in tokens if t != ""] # remove empty
#         vocab_counter.update(tokens)

In [17]:
# df.apply(lambda row: write_json(row), axis=1)

In [18]:
# if mode == 'train':
#     print("Writing vocab file...")
#     with open(os.path.join(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\data\fastAbs_data', "vocab_cnt.pkl"),'wb') as vocab_file:
#         pkl.dump(vocab_counter, vocab_file)
#     print("Finished writing vocab file")

In [19]:
# %history -g 

# Baseline

In [25]:
with open(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\BertSum\results\train_step6500.candidate','r', encoding='utf8') as f:
    pre = f.readlines()

with open(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\BertSum\results\train_step6500.gold','r', encoding='utf8') as f:
    gold = f.readlines()

In [46]:
dec = pd.DataFrame(columns=['predict', 'gold'])

In [55]:
dec['predict'] = pre
dec['gold'] = gold
dec['predict'] = dec['predict'].apply(lambda x: x.replace('\n','').replace('<q>',' '))
dec['gold'] = dec['gold'].apply(lambda x: x.replace('\n','').replace('<q>',' '))